In [ ]:
import scipy
import h5py
import skimage
from skimage.color import rgb2gray
from skimage import color
import torch
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
#import timm
import numpy as np
import io
from PIL import Image, ImageOps
from skimage import measure
from matplotlib import pyplot as plt
from skimage import filters

In [ ]:
def process_image(image):
    # prepare image for the model
    encoding = feature_extractor(image, return_tensors="pt")

    # forward pass
    with torch.no_grad():
        outputs = model(**encoding)
        predicted_depth = outputs.predicted_depth

    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=image.size[::-1],
        mode="bicubic",
        align_corners=False,
    ).squeeze()
    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(formatted)
    return img

    return result

In [ ]:
images = [Image.open(x) for x in ['catmask.jpg']]
iterator = 1
for i in images:
    widths, heights = i.size
    total_width = widths*2
    max_height = heights*2
    new_im = Image.new('RGBA', (total_width, max_height))
    grey = rgb2gray(i)
    edge_sobel = filters.sobel(grey)
    fig = plt.imshow(edge_sobel, cmap=plt.cm.gray)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png',bbox_inches='tight')
    im = Image.open(img_buf).resize(size=(widths, heights))
    Img = i.convert("RGBA")
    g = ImageOps.grayscale(i)
    grayRGB = ImageOps.colorize(g, black="black", white="white").convert("RGBA")

    new_im.paste(Img, (0,0))
    new_im.paste(grayRGB, (widths,0))
    new_im.paste(im, (0,heights))
    new_im.paste(i, (widths,heights))
    new_im.save('test.png')
    img_buf.close()

In [ ]:
# We will see if this works again